# Retrieval Augmented Generation
* **Created by:** Eric Martinez
* **For:** CSCI 4341
* **At:** University of Texas Rio-Grande Valley

## Step 0: Setup your `.env` file locally

Setup your `OPENAI_API_BASE` key and `OPENAI_API_KEY` in a file `.env` in this same folder.

```sh
# example .env contents (copy paste this into a .env file)
OPENAI_API_BASE=yourapibase
OPENAI_API_KEY=yourapikey
```

Install the required dependencies.

In [ ]:
%pip install -q -r requirements.txt

## Step 1: Create the Dataset

**Use your dataset from the previous homework.**

In [1]:
%%writefile data.json
[
    {
        "name": "Jason's Deli",
        "address": "1604 W University Dr."
    },
    {
        "name": "Taco Palenque",
        "address": "1414 W University Dr."
    },
    {
        "name": "University Drafthouse",
        "address": "2405 W University Dr. F"
    }
]

Writing data.json


Function to load the data

In [2]:
import json

def load_data():
    with open("data.json") as f:
        data = json.load(f)
    return data

Now actually load the data

In [3]:
data = load_data()
print(data)

[{'name': "Jason's Deli", 'address': '1604 W University Dr.'}, {'name': 'Taco Palenque', 'address': '1414 W University Dr.'}, {'name': 'University Drafthouse', 'address': '2405 W University Dr. F'}]


## Step 2: Create Chroma Collection

In [4]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
import os

import chromadb
from chromadb.utils import embedding_functions


def get_chroma_collection(collection_name):
    ## Use this one to save to memory
    # chroma_client = chromadb.Client() 

    ## Use this one to save to disk
    chroma_client = chromadb.PersistentClient(path=".")

    openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                    api_key=os.getenv("OPENAI_API_KEY"),
                    api_base=os.getenv("OPENAI_API_BASE"),
                    model_name="text-embedding-ada-002"
                )

    collection = chroma_client.get_or_create_collection(name=collection_name, embedding_function=openai_ef)
    return collection

In [5]:
collection = get_chroma_collection("food")

## Step 3: Add Data to Chroma Collection

In [6]:
def add_data_to_collection(data, collection):
    documents = []
    metadatas = []
    ids = []

    for i, restaurant in enumerate(data):
        name = restaurant['name']
        address = restaurant['address']

        # what are we embedding for each restaurant - obviously add to this
        embeddable_string = f"{name}"
        documents.append(embeddable_string)

        # lets just store everything we have as metadata
        metadatas.append(restaurant)

        # lets use the index as the id
        ids.append(str(i))

    collection.add(
        documents=documents,
        metadatas=metadatas,
        ids=ids
    )

In [7]:
add_data_to_collection(data, collection)

## Step 4: Query the Collection

In [8]:
def search_food(query):
    metadatas = []
    n_results = 2
    results = collection.query(query_texts=[query], n_results=2)
    
    for i in range(n_results):
        metadatas.append(results["metadatas"][0][i])
        
    return metadatas

In [9]:
results = search_food("fajita")

print(results)

[{'address': '1414 W University Dr.', 'name': 'Taco Palenque'}, {'address': '1604 W University Dr.', 'name': "Jason's Deli"}]


## Step 5: Build the Gradio UI for RAG

In [13]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
import gradio as gr
import openai
import re

# ---------------------------------------------------------------------------------------
def needs_tool(response):
    return "Tool:" in response

def extract_call(string):
    # regex pattern
    pattern = r'Tool: (\w+)\((.*?)\)'
    match = re.search(pattern, string)
    if match:
        tool_name = match.group(1)
        parameters = match.group(2).replace('"', '').split(', ')
        return tool_name, parameters
    else:
        return None, None
    
def invoke_tool(response):
    tool_name, parameters = extract_call(response)
    
    if tool_name == "search_food":
        tool_result = search_food(*parameters)
        
    return tool_result
# ---------------------------------------------------------------------------------------
# Define a function to get the AI's reply using the OpenAI API
def get_ai_reply(message, model="gpt-3.5-turbo", system_message=None, temperature=0, message_history=[]):
    # Initialize the messages list
    messages = []
    
    # Add the system message to the messages list
    if system_message is not None:
        messages += [{"role": "system", "content": system_message}]

    # Add the message history to the messages list
    if message_history is not None:
        messages += message_history
    
    if message is not None:
        # Add the user's message to the messages list
        messages += [{"role": "user", "content": message}]
    
    # Make an API call to the OpenAI ChatCompletion endpoint with the model and messages
    completion = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extract and return the AI's response from the API response
    return completion.choices[0].message.content.strip()
# ---------------------------------------------------------------------------------------
# Define a function to handle the chat interaction with the AI model
def chat(message, chatbot_messages, history_state):
    # Initialize chatbot_messages and history_state if they are not provided
    chatbot_messages = chatbot_messages or []
    history_state = history_state or []
    
    # Try to get the AI's reply using the get_ai_reply function
    try:
        prompt = """
        You are a helpful expert restaurant assistant named Jarvis.

        Your knowledge cut-off is: September 2021
        Today's date: October 18, 2023

        ## Tools

        You have access to the following tools:
        - search_food(query): Tool to lookup food based close to the user based on their query. Use this tools when the user is looking for food suggestions near them. You don't need to know the user's location, the tool doesn't need it. Be careful, the tool will return the top results in the database but not all of them may be relevant. Use your judgement when answering the user's question. Example: search_food("place to get a good burger and craft beer")

        ## Tool Rules

        When the user asks a question that can be answered by using a tool, you MUST do so. Do not answer from your training data.

        ## Using Tools

        To use a tool, reply with the following prefix "Tool: " then append the tool call (like a function call). 

        Behind the scenes, your software will pickup that you want to invoke a tool and invoke it for you and provide you the response.

        ## Using Tool Responses

        Answer the user's question using the response from the tool. Feel free to make it conversational. 
        """
        ai_reply = get_ai_reply(message, model="gpt-3.5-turbo", system_message=prompt.strip(), message_history=history_state)
            
        # Append the user's message and the AI's reply to the history_state list
        history_state.append({"role": "user", "content": message})
        history_state.append({"role": "assistant", "content": ai_reply})
        
        while(needs_tool(ai_reply)):
            tool_result = invoke_tool(ai_reply)
            history_state.append({"role": "assistant", "content": f"Tool Result: {tool_result}"})
            ai_reply = get_ai_reply(None, model="gpt-3.5-turbo", system_message=prompt.strip(), message_history=history_state)
            history_state.append({"role": "assistant", "content": ai_reply})
            
        # Append the user's message and the AI's reply to the chatbot_messages list for the UI
        chatbot_messages.append((message, ai_reply))

        # Return None (empty out the user's message textbox), the updated chatbot_messages, and the updated history_state
    except Exception as e:
        # If an error occurs, raise a Gradio error
        raise gr.Error(e)
        
    return None, chatbot_messages, history_state

# Define a function to launch the chatbot interface using Gradio
def get_chatbot_app():
    # Create the Gradio interface using the Blocks layout
    with gr.Blocks() as app:
        # Create a chatbot interface for the conversation
        chatbot = gr.Chatbot(label="Conversation")
        # Create a textbox for the user's message
        message = gr.Textbox(label="Message")
        # Create a state object to store the conversation history
        history_state = gr.State()
        # Create a button to send the user's message
        btn = gr.Button(value="Send")

        # Connect the send button to the chat function
        btn.click(chat, inputs=[message, chatbot, history_state], outputs=[message, chatbot, history_state])
        # Return the app
        return app
# ---------------------------------------------------------------------------------------        
# Call the launch_chatbot function to start the chatbot interface using Gradio
app = get_chatbot_app()
app.queue()  # this is to be able to queue multiple requests at once
app.launch(share=True)

Running on local URL:  http://127.0.0.1:7865
Running on public URL: https://62c91ef60d35887278.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
